In [1]:
import pandas as pd
import re

from sklearn.feature_extraction.text import CountVectorizer

import contractions

In [2]:
df = pd.read_parquet(r"C:\Users\fashaikh\Desktop\Thesis_main\sentiment_results\AfricanAmericanSouth_sentiment.parquet")

In [3]:
def pre_process(text):
    # Remove links
    text = re.sub(r'http://\S+|https://\S+', '', text)
    text = re.sub(r'http[s]?://\S+', '', text)
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"http", "", text)

    # Convert HTML references
    text = re.sub(r'&amp', 'and', text)
    text = re.sub(r'&lt', '<', text)
    text = re.sub(r'&gt', '>', text)
    
    # Remove new line characters
    text = re.sub(r'[\r\n]+', ' ', text)
    
    # Remove mentions
    text = re.sub(r'@\w+', '', text)
    
    # Remove hashtags
    text = re.sub(r'#\w+', '', text)

    # Remove multiple space characters
    text = re.sub(r'\s+',' ', text)
    
    # Convert to lowercase
    text = text.lower()
    return text

In [4]:
df['processed_text'] = df['cleanedContent'].apply(pre_process)

In [5]:
def expand_contractions(text):
  try:
    return contractions.fix(text)
  except:
    return text

In [6]:
df['processed_text'] = df['processed_text'].apply(expand_contractions)

In [7]:
def get_top_n_bigram(corpus, n=None):
    vec = CountVectorizer(ngram_range=(2, 2), stop_words='english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq = sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

In [ ]:
common_words = get_top_n_bigram(df['processed_text'], 50)

df1 = pd.DataFrame(common_words, columns = ['TweetText' , 'count'])
df1.groupby('TweetText').sum()['count'].sort_values(ascending=False).plot(
    kind='bar',
    title='Top 20 bigrams in Tweet before removing spams')

In [8]:
all_bigrams = get_top_n_bigram(df['processed_text'])